# TODO:


*   Add utilities to github
*   Update this notebook to clone repo
*   Add updates to this notbook to run a train & test for de-noising using images from google drive but utilities from github
*   Add in k-means & testing framework
*   Repeat with second notebook for hyper-spectral super sesolution



In [3]:
!git config --global user.name "RiverBotham"
!git config --global user.email "river.botham@gmail.com"
!git config --global user.password "MY_PASSWORD"

token = 'GIT_TOKEN'
username = 'RiverBotham'
repo = 'Raman'

!git clone https://{token}@github.com/{username}/{repo}

In [9]:
%cd {repo}/Denoising

/content/Raman/Denoising


In [13]:
# Imports
import os
import sys
import random
import datetime
import time
import shutil
import argparse
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import scipy.signal
import math
from skimage.metrics import structural_similarity as sk_ssim

import torch
from torch import nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributed as dist
import torch.utils.data.distributed
import torch.multiprocessing as mp
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, utils

import model, dataset, utilities

In [14]:
def train(dataloader, net, optimizer, scheduler, criterion, criterion_MSE, epoch, gpu, scheduler):

    batch_time = utilities.AverageMeter('Time', ':6.3f')
    losses = utilities.AverageMeter('Loss', ':.4e')
    psnr = utilities.AverageMeter('PSNR', ':.4f')
    ssim = utilities.AverageMeter('SSIM', ':.4f')
    progress = utilities.ProgressMeter(len(dataloader), [batch_time, psnr, ssim], prefix="Epoch: [{}]".format(epoch))

    end = time.time()
    for i, data in enumerate(dataloader):
        inputs = data['input_image']
        inputs = inputs.float()
        inputs = inputs.cuda(gpu)
        target = data['output_image']
        target = target.float()
        target = target.cuda(gpu)

        output = net(inputs)

        optimizer.zero_grad()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if scheduler == "cyclic-lr" or scheduler == "one-cycle-lr":
            scheduler.step()

        loss_MSE = criterion_MSE(output, target)
        losses.update(loss_MSE.item(), inputs.size(0))

        psnr_batch = utilities.calc_psnr(output, target)
        psnr.update(psnr_batch, inputs.size(0))

        ssim_batch = utilities.calc_ssim(output, target)
        ssim.update(ssim_batch, inputs.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

        if i % 20 == 0:
            progress.display(i)
    return losses.avg, psnr.avg, ssim.avg


In [15]:
def validate(dataloader, net, criterion_MSE, gpu):

    batch_time = utilities.AverageMeter('Time', ':6.3f')
    losses = utilities.AverageMeter('Loss', ':.4e')
    psnr = utilities.AverageMeter('PSNR', ':.4f')
    ssim = utilities.AverageMeter('SSIM', ':.4f')
    progress = utilities.ProgressMeter(len(dataloader), [batch_time, psnr, ssim], prefix='Validation: ')

    with torch.no_grad():
        end = time.time()
        for i, data in enumerate(dataloader):
            inputs = data['input_image']
            inputs = inputs.float()
            inputs = inputs.cuda(gpu)
            target = data['output_image']
            target = target.float()
            target = target.cuda(gpu)

            output = net(inputs)

            loss_MSE = criterion_MSE(output, target)
            losses.update(loss_MSE.item(), inputs.size(0))

            psnr_batch = utilities.calc_psnr(output, target)
            psnr.update(psnr_batch, inputs.size(0))

            ssim_batch = utilities.calc_ssim(output, target)
            ssim.update(ssim_batch, inputs.size(0))

            batch_time.update(time.time() - end)
            end = time.time()

            if i % 20 == 0:
                progress.display(i)

    return losses.avg, psnr.avg, ssim.avg

In [ ]:
def main_worker(gpu, ngpus_per_node, args, distributed, dist_url):


    if gpu is not None:
        print("Use GPU: {} for training".format(gpu))

    if distributed:
        if dist_url == "env://" and args.rank == -1:
            args.rank = int(os.environ["RANK"])
        if args.multiprocessing_distributed:
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend=args.dist_backend, init_method=dist_url,
                                world_size=args.world_size, rank=args.rank)

    # ----------------------------------------------------------------------------------------
    # Create model(s) and send to device(s)
    # ----------------------------------------------------------------------------------------
    scale = args.hr_image_size // args.lr_image_size
    net = model.Hyperspectral_RCAN(args.spectrum_len, scale).float()

    if distributed:
        if gpu is not None:
            torch.cuda.set_device(gpu)
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)

            net.cuda(gpu)
            net = torch.nn.parallel.DistributedDataParallel(net, device_ids=[gpu])
        else:
            net.cuda(gpu)
            net = torch.nn.parallel.DistributedDataParallel(net)
    elif gpu is not None:
        torch.cuda.set_device(gpu)
        net.cuda(gpu)
    else:
        net = nn.DataParallel(net).cuda()

    # ----------------------------------------------------------------------------------------
    # Define dataset path and data splits
    # ----------------------------------------------------------------------------------------
    dataset_path = "Dataset/"
    image_ids_csv = pd.read_csv(dataset_path + "Image_IDs.csv")

    image_ids = image_ids_csv["id"].values

    train_split = round(0.85 * len(image_ids))
    val_split = round(0.10 * len(image_ids))
    test_split = round(0.05 * len(image_ids))
    train_ids = image_ids[:train_split]
    val_ids = image_ids[train_split:train_split+val_split]
    test_ids = image_ids[train_split+val_split:]

    # ----------------------------------------------------------------------------------------
    # Create datasets and dataloaders
    # ----------------------------------------------------------------------------------------
    Raman_Dataset_Train = torch.utils.data.ConcatDataset([dataset.RamanImageDataset(train_ids, dataset_path, batch_size = args.batch_size,
                                                    hr_image_size = args.hr_image_size, lr_image_size = args.lr_image_size,
                                                    spectrum_len = args.spectrum_len, spectrum_shift = 0.1, spectrum_flip = True,
                                                    horizontal_flip = True, vertical_flip = True, rotate = True, patch = True, mixup = True),
                                                    dataset.RamanImageDataset(train_ids, dataset_path, batch_size = args.batch_size,
                                                    hr_image_size = args.hr_image_size, lr_image_size = args.lr_image_size,
                                                    spectrum_len = args.spectrum_len, spectrum_shift = -0.2, spectrum_flip = True,
                                                    horizontal_flip = True, vertical_flip = True, rotate = True, patch = True, mixup = True)])

    Raman_Dataset_Val = dataset.RamanImageDataset(val_ids, dataset_path, batch_size = args.batch_size,
                                                    hr_image_size = args.hr_image_size, lr_image_size = args.lr_image_size,
                                                    spectrum_len = args.spectrum_len)

    train_loader = DataLoader(Raman_Dataset_Train, batch_size = args.batch_size, shuffle = False, num_workers = args.workers)
    val_loader = DataLoader(Raman_Dataset_Val, batch_size = args.batch_size, shuffle = False, num_workers = args.workers)

    # ----------------------------------------------------------------------------------------
    # Define criterion(s), optimizer(s), and scheduler(s)
    # ----------------------------------------------------------------------------------------

    # ------------Criterion------------
    criterion = nn.L1Loss().cuda(gpu)
    criterion_MSE = nn.MSELoss().cuda(gpu)

    # ------------Optimizer------------
    if args.optimizer == "sgd":
        optimizer = optim.SGD(net.parameters(), lr = args.lr)
    elif args.optimizer == "adamW":
        optimizer = optim.AdamW(net.parameters(), lr = args.lr)
    else: # Adam
        optimizer = optim.Adam(net.parameters(), lr = args.lr)

    # ------------Scheduler------------
    if args.scheduler == "decay-lr":
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.2)
    elif args.scheduler == "multiplicative-lr":
        lmbda = lambda epoch: 0.985
        scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
    elif args.scheduler == "cyclic-lr":
        scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr = args.base_lr, max_lr = args.lr, mode = 'triangular2', cycle_momentum = False)
    elif args.scheduler == "one-cycle-lr":
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr = args.lr, steps_per_epoch=len(train_loader), epochs=args.epochs, cycle_momentum = False)
    else: # constant-lr
        scheduler = None

    print('Started Training')
    print('Training Details:')
    print('Network:         {}'.format(args.network))
    print('Epochs:          {}'.format(args.epochs))
    print('Batch Size:      {}'.format(args.batch_size))
    print('Optimizer:       {}'.format(args.optimizer))
    print('Scheduler:       {}'.format(args.scheduler))
    print('Learning Rate:   {}'.format(args.lr))
    print('Spectrum Length: {}'.format(args.spectrum_len))

    date = datetime.datetime.now().strftime("%Y_%m_%d")

    log_dir = "runs/{}_{}_{}_{}_{}x".format(date, args.optimizer, args.scheduler, args.network, scale)
    models_dir = "{}_{}_{}_{}_{}x.pt".format(date, args.optimizer, args.scheduler, args.network, scale)

    writer = SummaryWriter(log_dir = log_dir)

    for epoch in range(args.epochs):
        train_loss, train_psnr, train_ssim = train(train_loader, net, optimizer, scheduler, criterion, criterion_MSE, epoch, args)
        valid_loss, valid_psnr, valid_ssim = validate(val_loader, net, criterion_MSE, args)
        if args.scheduler != "cyclic-lr" and args.scheduler != "one-cycle-lr" and args.scheduler != "constant-lr":
            scheduler.step()

        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', valid_loss, epoch)
        writer.add_scalar('PSNR/train', train_psnr, epoch)
        writer.add_scalar('PSNR/val', valid_psnr, epoch)
        writer.add_scalar('SSIM/train', train_ssim, epoch)
        writer.add_scalar('SSIM/val', valid_ssim, epoch)

    torch.save(net.state_dict(), models_dir)
    print('Finished Training')